In [ ]:
# using Pkg
# Pkg.activate("..") 
# Pkg.instantiate()
using JPEC, Plots
gr() 

  Activating project at `~/Github/JPEC`


Plots.GRBackend()

In [6]:
using LinearAlgebra
using FFTW
using Printf
using Plots

### read Equil

In [7]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
equil_input = JPEC.Equilibrium.EquilInput(
    "beta_1.00",        # eq_filename
    "efit",          # eq_type
    "boozer",        # jac_type
    0.01,             # psilow
    1.0,             # psihigh
    100,             # mpsi (number of radial grid points)
    128              # mtheta (number of poloidal grid points)
)

# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_input)
println("Equilibrium reconstruction complete.")

Starting equilibrium reconstruction...
--- Julia Equilibrium Setup ---
Equilibrium file: beta_1.00
Type = efit, Jac_type = boozer
----------------------------------------
--> Processing EFIT g-file: beta_1.00
--> Parsed from header: nw=129, nh=128
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 3.115238, Z = 0.000008, |ΔR|=1.15e-01, |ΔZ|=1.11e-05
  Iter  2: R = 3.117635, Z = -0.000003, |ΔR|=2.40e-03, |ΔZ|=1.10e-05
  Iter  3: R = 3.117636, Z = -0.000003, |ΔR|=1.07e-06, |ΔZ|=1.04e-09
  Iter  4: R = 3.117636, Z = -0.000003, |ΔR|=2.12e-13, |ΔZ|=5.62e-16
Magnetic axis found at R=3.1176357129137466, Z=-2.9184696943092166e-6.
Finding inboard separatrix crossing...
  Restart attempt 1/6 with initial R = 2.675802
inboard separatrix found at R=1.5000175239830411.
Finding outboard separ

### Use spline_eval 
sq!!

In [ ]:
# The plasma_eq object contains the final 1D profile spline, `sq`.
# We will evaluate it on a fine grid to get smooth plot lines.
psi_norm_grid = range(0.0, 1.0, length=200)

# spline_eval returns the function values (and derivatives if requested).
# For sq, the quantities are: 1:F, 2:P*mu0, 3:Toroidal Flux, 4:q
f_profiles = JPEC.SplinesMod.spline_eval(plasma_eq., collect(psi_norm_grid))

# Extract each profile into its own variable for clarity
F_profile      = f_profiles[:, 1]
P_profile      = f_profiles[:, 2]
tor_flux_prof  = f_profiles[:, 3]
q_profile      = f_profiles[:, 4];

In [55]:
JPEC.SplinesMod.spline_eval(plasma_eq.sq, psi_norm_grid[10])[4]

10.16821181688277

In [113]:
"""
sing_find(mex, nn, plasma_eq; itmax=200, nsing=1000)

Finds rational surfaces (where m = n*q) in a plasma equilibrium profile.
- Uses binary search between intervals defined in psi.
- Returns NamedTuples with (m, q, q1, psifac, rho).

Arguments:
  mex::Int           : Number of q profile intervals/extrema (count + 1; e.g. length of qex and psiex)
  nn::Int            : n-number (toroidal mode)
  plasma_eq          : Should support, e.g., JPEC.SplinesMod.spline_eval(plasma_eq.sq, psi, deriv)
  itmax::Int         : Maximum number of bisection steps (default 200)
  nsing::Int         : Max number of rational surfaces (default 1000)
Returns:
  surfaces::Vector{NamedTuple}
"""
function sing_find(mex, nn, plasma_eq; itmax=200, nsing=1000)
    # Setup storage for the singular surfaces found
    surfaces = NamedTuple[]
    # Choose a set of search points for psi (resolution much higher than number of extrema!)
    psigrid = range(0.0, 1.0, length=200) |> collect

    # Evaluate q(psi) at all grid points
    qvals = [JPEC.SplinesMod.spline_eval(plasma_eq.sq, psi, 0)[4] for psi in psigrid]

    # Loop over intervals (each extrema is a pair of adjacent grid points)
    for iex in 2:mex
        
        # For each interval between psi[i-1] and psi[i]:
        # --- Find the values at endpoints
        psi0, psi1 = psigrid[iex-1], psigrid[iex]
        q0,   q1   = qvals[iex-1], qvals[iex]
        dq = q1 - q0

        println("Processing interval $iex: psi0=$(psi0), psi1=$(psi1), q0=$(q0), q1=$(q1), dq=$(dq)")

        # Skip intervals with no range or (dangerously) no sign change
        if abs(dq) < 1e-10
            continue
        end
        
        # Compute m at psi0 for this interval, follow DCON convention for m and step direction dm
        m = round(Int, nn * q0)
        if dq > 0
            m += 1
        end
        dm = sign(dq * nn)
        
        # The interval in m values: as long as m is between m0=nn*q0 and m1=nn*q1 (inclusive):
        while (m - nn * q0)*(m - nn * q1) <= 0            # while m is in [nn*q0, nn*q1]
            # Root-finding for this m: search psi in [psi0, psi1] where m = nn * q(psi)
            low, high = psi0, psi1        # bisection interval
            it = 0
            found = false
            # Bisection to solve m = nn*q(psi)
            while it < itmax
                it += 1
                mid = 0.5*(low+high)
                qmid = JPEC.SplinesMod.spline_eval(plasma_eq.sq, mid, 0)[4]
                singfac = (m - nn*qmid)*dm
                # Root cross found if singfac nearly zero
                if abs(singfac) < 1e-12
                    found = true
                    break
                # Decide which half to keep
                elseif singfac > 0
                    low = mid
                else
                    high = mid
                end
                # If width is tiny, accept
                if abs(high-low) < 1e-12
                    found = true
                    break
                end
            end
            if !found
                @warn "sing_find: bisection did not converge for m=$m in [psi=$(low),psi=$(high)]"
                break
            end

            # Evaluate q' (derivative) at the found surface
            q1val = JPEC.SplinesMod.spline_eval(plasma_eq.sq, mid, 1)[4]

            # Store this singular surface
            push!(surfaces, (
                m       = m,
                q       = m/nn,
                q1      = q1val,
                psifac  = mid,
                rho     = sqrt(mid)
            ))
            # If we reach nsing, bail out
            if length(surfaces) >= nsing
                @warn "sing_find: reached nsing=$nsing surfaces, stopping early."
                return surfaces
            end
            # Move to next m in direction dm
            m += dm
        end
    end
    return surfaces
end

sing_find

In [114]:
sing_find(20, 1, plasma_eq)

Processing interval 2: psi0=0.0, psi1=0.005025125628140704, q0=10.117814093315939, q1=10.18941109902143, dq=0.07159700570549177
Processing interval 3: psi0=0.005025125628140704, psi1=0.010050251256281407, q0=10.18941109902143, q1=10.1970473696912, dq=0.007636270669769019
Processing interval 4: psi0=0.010050251256281407, psi1=0.01507537688442211, q0=10.1970473696912, q1=10.192122777369232, dq=-0.004924592321968291
Processing interval 5: psi0=0.01507537688442211, psi1=0.020100502512562814, q0=10.192122777369232, q1=10.187786602670085, dq=-0.004336174699146156
Processing interval 6: psi0=0.020100502512562814, psi1=0.02512562814070352, q0=10.187786602670085, q1=10.183322591217255, dq=-0.004464011452830263
Processing interval 7: psi0=0.02512562814070352, psi1=0.03015075376884422, q0=10.183322591217255, q1=10.179364571781797, dq=-0.003958019435458482
Processing interval 8: psi0=0.03015075376884422, psi1=0.035175879396984924, q0=10.179364571781797, q1=10.175409279374447, dq=-0.003955292407349

NamedTuple[]